THIS NOTEBOOK PERFORM THE FOLLOWING:
- READ IMAGES .ZIP FILES FROM GOOGLE CLOUD BUCKET
- ROTATE THE IMAGES IN ORDER TO FURTHER READ THE TEXT HORIZONTALLY
- OCR: CONVERT IMAGES TO TEXT THROUGH TESSERACT API
- RECORD THE TEXT FILE TO CURRENT GOOGLE DRIVE

In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (5,401 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [ ]:
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14072 sha256=c4dabffda34ef5268b2a6a78e02e2caa1ee22c465c30543952db14d131014875
  Stored in directory: /root/.cache/pip/wheels/a4/89/b9/3f11250225d0f90e5454fcc30fd1b7208db226850715aa9ace
Successfully built pytesseract


In [ ]:
import os
import pytesseract
import zipfile
from PIL import Image
import re

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'google_cloud_project_id'

In [ ]:
#!gcloud config set project {project_id}

In [ ]:
!gsutil cp -n gs://'bucket_name'/'zip_file' /content

Skipping existing item: file:///content/estatutos-png.zip


In [ ]:
rutazip = 'google_drive_zip_path'
zipimg = zipfile.ZipFile(rutazip)

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
!sudo apt-get install tesseract-ocr-spa
#pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/pytesseract'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-spa is already the newest version (4.00~git24-0e00fe6-1.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
def rotar(imagen):
  try:
    osd_rotated_image = pytesseract.image_to_osd(imagen)
    angle_rotated_image = re.search('(?<=Rotate: )\d+', osd_rotated_image).group(0)
    orientation_confidence = re.search('(?<=Orientation confidence: )\d+', osd_rotated_image).group(0)
    if (angle_rotated_image == '0') or (orientation_confidence < 9) :
      imagen_rotada = imagen
      print('no rotado')
      print('orientation confidence: ',orientation_confidence)
      # break the loop once we get the correctly deskewed image
    elif (angle_rotated_image == '90'):
      imagen_rotada = imagen.rotate(90)
      print('rotado 90 grados')
      print('orientation confidence: ',orientation_confidence)
       # rotate(image,angle,background_color)
    elif (angle_rotated_image == '180'):
      imagen_rotada = imagen.rotate(180)
      print('rotado 180 grados')
      print('orientation confidence: ',orientation_confidence)
    elif (angle_rotated_image == '270'):
      imagen_rotada = imagen.rotate(270)
      print('rotado 270 grados')
      print('orientation confidence: ',orientation_confidence)
  except:
    imagen_rotada = imagen
    print("Exception:")
    print("Error en lectura de caracteres. Posible imagen vacía")
  finally:
    return imagen_rotada
  

In [ ]:
def primerparte(nombre):
  punto = nombre.find('.')
  pparte = nombre[0:punto]
  return pparte

In [ ]:
custom_config = r'-l spa --oem 3 --psm 6'
largo = len(zipimg.namelist())
#texto = []
mtexto = ""
nombreanterior = ""
n = len(zipimg.namelist())-1
for x in range(0,n):
  nombrearchivo = zipimg.namelist()[x]
  if nombrearchivo[1] != "_":
    print(nombrearchivo)
    abierto = zipimg.open(nombrearchivo)
    img = Image.open(abierto)
    img = rotar(img)
    if primerparte(nombrearchivo) == primerparte(nombreanterior):
      mtexto = mtexto + pytesseract.image_to_string(img, config=custom_config)
      mtexto = mtexto.replace("\n", " ")

    else:
      if mtexto != "":
        file = open(primerparte(nombreanterior)+ '.txt', "w")
        file.write(mtexto)
        file.close()
        nombretemp = primerparte(nombreanterior) + '.txt'
        !gsutil cp -n $nombretemp gs://'archivos_crudos'
        print('se escribe archivo: ' + primerparte(nombreanterior) + '.txt' )
      mtexto = pytesseract.image_to_string(img, config=custom_config)    
      mtexto = mtexto.replace("\n", " ")
      print('new mtexto init')
   
    nombreanterior = nombrearchivo

# Se escribe el último archivo si la lista no está vacía
if nombreanterior != "":  
  file = open(primerparte(nombreanterior)+ '.txt', "w")
  file.write(mtexto)
  file.close()
  nombretemp = primerparte(nombreanterior) + '.txt'
  !gsutil cp -n $nombretemp gs://'archivos_crudos'
  print('record new file: ' + primerparte(nombreanterior) + '.txt' )